In [1]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

data = pd.read_csv(r'D:\grp_project_final.csv')

In [2]:
data.head()

,activity,emotion,timestamp,TEMP_1,EDA_1,BVP_1,HR_1,IBI_1
0,Running,Energetic,00:00:00,31.45,0.481635,-0.0,103.00,83.359375
1,Eating,Content,00:30:00,31.45,0.595640,-0.0,98.33,84.125000
2,Sleeping,Calm,01:00:00,31.45,0.611011,-0.0,96.00,103.296875
3,Walking,Nostalgic,01:30:00,31.45,0.627663,-0.0,92.80,109.328125
4,Studying,Focused,02:00:00,31.45,0.672496,-0.0,88.00,110.312500


In [3]:
data.shape

(24000, 8)

In [4]:
#Extracting hours from the timestamp column.
data['Hour'] = pd.to_datetime(data['timestamp']).dt.hour

In [5]:
#Segregating hours into night, morning, afternoon and evening.
data['TimeOfDay'] = pd.cut(data['Hour'], bins=[-1, 6, 12, 18, 24], labels=['night', 'morning', 'afternoon', 'evening'])

In [6]:
#Using labelEncoder to convert repeating strings to numerical representations.
time_encoder = LabelEncoder()
data['TimeOfDay'] = time_encoder.fit_transform(data['TimeOfDay'])
activity_encoder = LabelEncoder()
emotion_encoder = LabelEncoder()
data['ActivitywithLE'] = activity_encoder.fit_transform(data['activity'])
data['EmotionwithLE'] = emotion_encoder.fit_transform(data['emotion'])

In [7]:
#X contains 4 parameters - Daytime, Activity, Body temperature in celcius and electrodermal activity in microsiemens.
X = data[['TimeOfDay', 'ActivitywithLE', 'TEMP_1', 'EDA_1','BVP_1','HR_1','IBI_1']]
#y contains target column, Emotion.
y = data['EmotionwithLE']

#Splitting the data into train and test sets (80:20).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

In [8]:
#Using StandardScaler to scale all parameters.
scaler = StandardScaler()
X_train[['TimeOfDay', 'ActivitywithLE', 'TEMP_1', 'EDA_1','BVP_1','HR_1','IBI_1']] = scaler.fit_transform(X_train[['TimeOfDay', 'ActivitywithLE', 'TEMP_1', 'EDA_1','BVP_1','HR_1','IBI_1']])
X_test[['TimeOfDay', 'ActivitywithLE', 'TEMP_1', 'EDA_1','BVP_1','HR_1','IBI_1']] = scaler.transform(X_test[['TimeOfDay', 'ActivitywithLE', 'TEMP_1', 'EDA_1','BVP_1','HR_1','IBI_1']])

In [11]:
#Converting data to PyTorch tensors.
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)
X_train_tensor = X_train_tensor.unsqueeze(1)
X_test_tensor = X_test_tensor.unsqueeze(1)
print(X_train_tensor.shape)


torch.Size([19200, 1, 7])


In [12]:

class MoodLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MoodLSTM, self).__init__()
        self.hidden_dim = hidden_dim  # Store hidden_dim as a class attribute
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(1, batch_size, self.hidden_dim).requires_grad_()
        c0 = torch.zeros(1, batch_size, self.hidden_dim).requires_grad_()
        out, _ = self.lstm(x, (h0.detach(), c0.detach()))  
        out = self.fc(out[:, -1, :])
        return out

#Setting model architecture.
input_dim = 7  
hidden_dim = 64
output_dim = 36 #Number of classes in Emotion column.

#Instantiating the model.
model = MoodLSTM(input_dim, hidden_dim, output_dim)

#Defining loss function and Optimizer.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#Training the model.
epochs = 400
for epoch in range(epochs):
    #Forward pass.
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    #Backward pass.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, loss.item()))




Epoch [1/400], Loss: 3.5990
Epoch [2/400], Loss: 3.5966
Epoch [3/400], Loss: 3.5943
Epoch [4/400], Loss: 3.5919
Epoch [5/400], Loss: 3.5896
Epoch [6/400], Loss: 3.5872
Epoch [7/400], Loss: 3.5849
Epoch [8/400], Loss: 3.5825
Epoch [9/400], Loss: 3.5801
Epoch [10/400], Loss: 3.5777
Epoch [11/400], Loss: 3.5753
Epoch [12/400], Loss: 3.5729
Epoch [13/400], Loss: 3.5704
Epoch [14/400], Loss: 3.5679
Epoch [15/400], Loss: 3.5654
Epoch [16/400], Loss: 3.5629
Epoch [17/400], Loss: 3.5603
Epoch [18/400], Loss: 3.5576
Epoch [19/400], Loss: 3.5550
Epoch [20/400], Loss: 3.5522
Epoch [21/400], Loss: 3.5495
Epoch [22/400], Loss: 3.5466
Epoch [23/400], Loss: 3.5438
Epoch [24/400], Loss: 3.5408
Epoch [25/400], Loss: 3.5379
Epoch [26/400], Loss: 3.5348
Epoch [27/400], Loss: 3.5317
Epoch [28/400], Loss: 3.5285
Epoch [29/400], Loss: 3.5253
Epoch [30/400], Loss: 3.5220
Epoch [31/400], Loss: 3.5187
Epoch [32/400], Loss: 3.5152
Epoch [33/400], Loss: 3.5117
Epoch [34/400], Loss: 3.5082
Epoch [35/400], Loss: 3

In [13]:
#Testing the model for accuracy.
with torch.no_grad():
    outputs = model(X_test_tensor)

_, predicted = torch.max(outputs, 1)
correct = (predicted == y_test_tensor).sum().item()
total = y_test_tensor.size(0)
accuracy = correct / total

print('Accuracy on test data: {:.2f}%'.format(accuracy * 100))


Accuracy on test data: 9.04%
